In [5]:
from langchain_google_genai import ChatGoogleGenerativeAI
import google.generativeai as genai
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage
import requests
import os

In [2]:
# Load environment variables from the .env file
load_dotenv()

True

In [3]:

# # Access the variables
gemini_api_key = os.getenv("GEMINI_API_KEY")

In [9]:
# using langchain_google_genai
gemini_model = ChatGoogleGenerativeAI(model='gemini-1.5-flash',google_api_key=gemini_api_key)
result = gemini_model.invoke('What is the capital of India')

print(result.content)

The capital of India is **New Delhi**.


In [6]:
# tool create

@tool
def multiply(a: int, b: int) -> int:
  """Given 2 numbers a and b this tool returns their product"""
  return a * b

In [7]:
print(multiply.invoke({'a':3, 'b':4}))

12


In [8]:
print(multiply.name)
print(multiply.description)
print(multiply.input_schema)
print(multiply.args)

multiply
Given 2 numbers a and b this tool returns their product
<class 'langchain_core.utils.pydantic.multiply'>
{'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}


### Tool Binding with LLM

In [10]:
llm_with_tools = gemini_model.bind_tools([multiply])

In [12]:
llm_with_tools.invoke('Hi how are you')

AIMessage(content="I'm doing well, thank you for asking! How are you today?", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run--ed79d28e-bbf4-4e7e-a65c-3abd59f8de2a-0', usage_metadata={'input_tokens': 24, 'output_tokens': 17, 'total_tokens': 41, 'input_token_details': {'cache_read': 0}})

In [13]:
query = HumanMessage('can you multiply 3 with 1000')

In [14]:
messages = [query]

In [15]:
messages

[HumanMessage(content='can you multiply 3 with 1000', additional_kwargs={}, response_metadata={})]

In [16]:
result = llm_with_tools.invoke(messages)

In [17]:
messages.append(result)

In [18]:
messages

[HumanMessage(content='can you multiply 3 with 1000', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'function_call': {'name': 'multiply', 'arguments': '{"a": 3.0, "b": 1000.0}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run--95f2ecbc-42b4-4bbd-8120-8165149cf879-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3.0, 'b': 1000.0}, 'id': '06d0ef9b-72a4-4b91-bad7-82af2fda6f52', 'type': 'tool_call'}], usage_metadata={'input_tokens': 31, 'output_tokens': 5, 'total_tokens': 36, 'input_token_details': {'cache_read': 0}})]

In [19]:
tool_result = multiply.invoke(result.tool_calls[0])

In [21]:
result.tool_calls[0]

{'name': 'multiply',
 'args': {'a': 3.0, 'b': 1000.0},
 'id': '06d0ef9b-72a4-4b91-bad7-82af2fda6f52',
 'type': 'tool_call'}

In [20]:
tool_result

ToolMessage(content='3000', name='multiply', tool_call_id='06d0ef9b-72a4-4b91-bad7-82af2fda6f52')

In [22]:
messages.append(tool_result)

In [23]:
messages

[HumanMessage(content='can you multiply 3 with 1000', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'function_call': {'name': 'multiply', 'arguments': '{"a": 3.0, "b": 1000.0}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run--95f2ecbc-42b4-4bbd-8120-8165149cf879-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3.0, 'b': 1000.0}, 'id': '06d0ef9b-72a4-4b91-bad7-82af2fda6f52', 'type': 'tool_call'}], usage_metadata={'input_tokens': 31, 'output_tokens': 5, 'total_tokens': 36, 'input_token_details': {'cache_read': 0}}),
 ToolMessage(content='3000', name='multiply', tool_call_id='06d0ef9b-72a4-4b91-bad7-82af2fda6f52')]

In [24]:
llm_with_tools.invoke(messages).content

'3 multiplied by 1000 is 3000.'

### Currency conversion tool

In [25]:
# tool create
from langchain_core.tools import InjectedToolArg
from typing import Annotated

@tool
def get_conversion_factor(base_currency: str, target_currency: str) -> float:
  """
  This function fetches the currency conversion factor between a given base currency and a target currency
  """
  url = f'https://v6.exchangerate-api.com/v6/c754eab14ffab33112e380ca/pair/{base_currency}/{target_currency}'

  response = requests.get(url)

  return response.json()

@tool
def convert(base_currency_value: int, conversion_rate: Annotated[float, InjectedToolArg]) -> float:
  """
  given a currency conversion rate this function calculates the target currency value from a given base currency value
  """

  return base_currency_value * conversion_rate


In [26]:
convert.args

{'base_currency_value': {'title': 'Base Currency Value', 'type': 'integer'},
 'conversion_rate': {'title': 'Conversion Rate', 'type': 'number'}}

In [27]:
get_conversion_factor.invoke({'base_currency':'USD','target_currency':'INR'})

{'result': 'success',
 'documentation': 'https://www.exchangerate-api.com/docs',
 'terms_of_use': 'https://www.exchangerate-api.com/terms',
 'time_last_update_unix': 1754352001,
 'time_last_update_utc': 'Tue, 05 Aug 2025 00:00:01 +0000',
 'time_next_update_unix': 1754438401,
 'time_next_update_utc': 'Wed, 06 Aug 2025 00:00:01 +0000',
 'base_code': 'USD',
 'target_code': 'INR',
 'conversion_rate': 87.7131}

In [28]:
convert.invoke({'base_currency_value':10, 'conversion_rate':85.16})

851.5999999999999

In [29]:
llm_with_tools = gemini_model.bind_tools([get_conversion_factor, convert])

In [35]:
llm_with_tools

RunnableBinding(bound=ChatGoogleGenerativeAI(model='models/gemini-1.5-flash', google_api_key=SecretStr('**********'), client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x10ea324d0>, default_metadata=()), kwargs={'tools': [{'type': 'function', 'function': {'name': 'get_conversion_factor', 'description': 'This function fetches the currency conversion factor between a given base currency and a target currency', 'parameters': {'properties': {'base_currency': {'type': 'string'}, 'target_currency': {'type': 'string'}}, 'required': ['base_currency', 'target_currency'], 'type': 'object'}}}, {'type': 'function', 'function': {'name': 'convert', 'description': 'given a currency conversion rate this function calculates the target currency value from a given base currency value', 'parameters': {'properties': {'base_currency_value': {'type': 'integer'}}, 'required': ['base_currency_value'], 'type': 'object'}}}]}, config={}, config_factor

In [30]:
messages = [HumanMessage('What is the conversion factor between INR and USD, and based on that can you convert 10 inr to usd')]

In [31]:
messages

[HumanMessage(content='What is the conversion factor between INR and USD, and based on that can you convert 10 inr to usd', additional_kwargs={}, response_metadata={})]

In [32]:
ai_message = llm_with_tools.invoke(messages)

In [33]:
messages.append(ai_message)

In [36]:
ai_message.tool_calls

[]

In [37]:
import json

for tool_call in ai_message.tool_calls:
  # execute the 1st tool and get the value of conversion rate
  if tool_call['name'] == 'get_conversion_factor':
    tool_message1 = get_conversion_factor.invoke(tool_call)
    # fetch this conversion rate
    conversion_rate = json.loads(tool_message1.content)['conversion_rate']
    # append this tool message to messages list
    messages.append(tool_message1)
  # execute the 2nd tool using the conversion rate from tool 1
  if tool_call['name'] == 'convert':
    # fetch the current arg
    tool_call['args']['conversion_rate'] = conversion_rate
    tool_message2 = convert.invoke(tool_call)
    messages.append(tool_message2)

In [38]:
messages

[HumanMessage(content='What is the conversion factor between INR and USD, and based on that can you convert 10 inr to usd', additional_kwargs={}, response_metadata={}),
 AIMessage(content='I need more information.  The available function `get_conversion_factor` requires `base_currency` and `target_currency` to be specified.  I also need the conversion factor to be returned from that function before I can use the `convert` function.  There is no data provided to populate these values.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run--ecebfd94-8add-4f14-8b6e-357eb22d613b-0', usage_metadata={'input_tokens': 92, 'output_tokens': 67, 'total_tokens': 159, 'input_token_details': {'cache_read': 0}})]

In [39]:
llm_with_tools.invoke(messages).content

'To get the conversion factor and perform the conversion, please provide the necessary data or a way to obtain it.'

In [40]:
from langchain.agents import initialize_agent, AgentType

# Step 5: Initialize the Agent ---
agent_executor = initialize_agent(
    tools=[get_conversion_factor, convert],
    llm=gemini_model,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,  # using ReAct pattern
    verbose=True  # shows internal thinking
)

/var/folders/np/0_9d0fln6jd4jpfr_3m4mw280000gn/T/ipykernel_31327/1482899919.py:4: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent_executor = initialize_agent(


In [41]:
# --- Step 6: Run the Agent ---
user_query = "Hi how are you?"

response = agent_executor.invoke({"input": user_query})



> Entering new AgentExecutor chain...
Thought:I am doing well, thank you for asking.  I am ready to assist you.  I will respond directly since there is no calculation or tool needed.

Action:
```json
{
  "action": "Final Answer",
  "action_input": "I am doing well, thank you for asking! How can I help you today?"
}
```

> Finished chain.
